## Iteration using Cursors

---
### Bound cursors

In [28]:
DO $$
DECLARE 
    cur CURSOR FOR SELECT * from customers;
    rc  INT = 3;
BEGIN
    FOR rec in cur LOOP
        IF rc > 0 THEN
            RAISE INFO '(%, %)', rec.customerid, rec.firstname;
        END IF;            
        rc = rc - 1;
        IF rc = 0 THEN 
            EXIT;
        END IF;
    END LOOP;
END;
$$  

---
### Unbound cursor

In [29]:
DO $$
DECLARE 
    cur REFCURSOR;
    fn customers.firstname%TYPE;
BEGIN
    OPEN cur FOR SELECT firstname from customers;
    FOR rc IN 1..3 LOOP
        FETCH NEXT FROM cur INTO fn;
        RAISE INFO '(%, %)', rc, fn;
        rc = rc + 1;
    END LOOP;
    CLOSE cur;
END;
$$    

---
### Cursor with parameter

In [30]:
CREATE OR REPLACE FUNCTION get_cur(prefix VARCHAR)
    RETURNS refcursor
LANGUAGE plpgsql AS $$
DECLARE cur CURSOR (p VARCHAR) FOR 
    SELECT * from customers WHERE firstname LIKE p || '%';
BEGIN
    OPEN cur(prefix);
    RETURN cur;
END;
$$;

In [31]:
BEGIN;  
SELECT get_cur('AZ');
FETCH 5 FROM cur;
COMMIT;

---
### Clean up

In [32]:
DROP FUNCTION IF EXISTS get_cur(prefix VARCHAR, refcursor);
DROP FUNCTION IF EXISTS get_cur(prefix VARCHAR);